In [1]:
%%bash
ls

1.dataSetup.ipynb
2.CNNLSTM.ipynb
3. kerasOnlyExecution.ipynb
4. Subject Specific Classification.ipynb
5. Autoencoder.ipynb
__pycache__
history.pkl
history3.pkl
history4.pkl
scratch.ipynb
tf_mapper.py
tfrecords.py


In [12]:
import tensorflow as tf
# tf.enable_eager_execution()
import tf_mapper as tfm

def read_tfrecord(example):
    features = { \
                'data':  tf.io.FixedLenFeature([1500*3], tf.float32,),\
                'on_off':  tf.io.FixedLenFeature([1], tf.int64,),\
                'dyskinesia':  tf.io.FixedLenFeature([1], tf.int64,),
                'measurement_id':  tf.io.FixedLenFeature([1], tf.int64,),\
                'tremor':  tf.io.FixedLenFeature([1], tf.int64,), \
                'age':  tf.io.FixedLenFeature([1], tf.int64,), \
                "subjects": tf.io.FixedLenFeature([1], tf.int64), \
                "gender": tf.io.FixedLenFeature([1], tf.int64), \
                "UPDRS_PartI_Total": tf.io.FixedLenFeature([1], tf.int64), \
                "UPDRS_PartII_Total": tf.io.FixedLenFeature([1], tf.int64), \
                "UPDRS_4.1": tf.io.FixedLenFeature([1], tf.int64), \
                "UPDRS_4.2": tf.io.FixedLenFeature([1], tf.int64), \
                "UPDRS_4.3": tf.io.FixedLenFeature([1], tf.int64), \
                "UPDRS_4.4": tf.io.FixedLenFeature([1], tf.int64), \
                "UPDRS_4.5": tf.io.FixedLenFeature([1], tf.int64), \
                "UPDRS_4.6": tf.io.FixedLenFeature([1], tf.int64)
               }

    example = tf.io.parse_single_example(example, features)
    return example
def map_example_to_simple_with_mid(example):
    data = example['data']
    data = tf.reshape(data, (1500,3))
    return data, example['measurement_id']
def tf_is_in_set(a, b):
    return tf.reduce_sum(tf.cast(tf.equal(b, a), tf.int64)) >= 1
#https://github.com/matthew-brett/transforms3d/blob/master/transforms3d/axangles.py
def tfaxangle2mat(x, y, z, angle, is_normalized=False):
#     x, y, z = axis
    if not is_normalized:
        n = tf.math.sqrt(x*x + y*y + z*z)
        x = x/n
        y = y/n
        z = z/n
    c = tf.math.cos(angle); s = tf.math.sin(angle); C = 1-c
    xs = x*s;   ys = y*s;   zs = z*s
    xC = x*C;   yC = y*C;   zC = z*C
    xyC = x*yC; yzC = y*zC; zxC = z*xC
    return tf.reshape(tf.concat([
             x*xC+c,   xyC-zs,   zxC+ys ,
             xyC+zs,   y*yC+c,   yzC-xs ,
             zxC-ys,   yzC+xs,   z*zC+c ], axis=-1), (3,3))
std = 1/4 #allow deviation from real rotation with pi/4 std
def map_example_to_simple(example):
    data = example["data"]
    data = tf.reshape(data, (1500,3))
    update_matrix = tfaxangle2mat(tf.constant(0.0), tf.constant(0.0), tf.constant(1.0), tf.random.normal((1,)) * tf.constant(3.14*std))
    update_matrix = update_matrix @ tfaxangle2mat(tf.constant(0.0), tf.constant(1.0), tf.constant(0.0), tf.random.normal((1,)) * tf.constant(3.14*std))
    update_matrix = update_matrix @ tfaxangle2mat(tf.constant(1.0), tf.constant(0.0), tf.constant(0.0), tf.random.normal((1,)) * tf.constant(3.14*std))
    data = data @ update_matrix
    return data, data
def get_batched_dataset(filenames, batch_size, m_ids, max_queue_size=10,  n_process=4, is_train=False):
    option_no_order = tf.data.Options()
    option_no_order.experimental_deterministic = False

    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.with_options(option_no_order)
    dataset = dataset.interleave(tf.data.TFRecordDataset, cycle_length=1, num_parallel_calls=1)

    dataset = dataset.map(read_tfrecord, num_parallel_calls=n_process)
    dataset = dataset.filter(lambda example: tf_is_in_set(example["measurement_id"][0], tf.constant(m_ids, dtype=tf.int64)))
    dataset = dataset.filter(lambda example:  tf.math.logical_not(tf.reduce_any(tf.math.is_nan(example["data"]))))
    dataset = dataset.map(map_example_to_simple)
    dataset = dataset.repeat()
    if is_train:
        dataset = dataset.shuffle(2056)
#     dataset = dataset.batch(batch_size, drop_remainder=True)
    if is_train:
        dataset = dataset.prefetch(max_queue_size)
    else:
        dataset = dataset.prefetch(int(max_queue_size/4)) #store a lot less for the other sets to avoid wasting memory
    dataset = dataset.batch(batch_size, drop_remainder=True)
    return dataset

In [6]:
labels = tfm.get_labels()

In [7]:
labels.shape

(1858, 5)

In [8]:
from sklearn.model_selection import train_test_split
train_mid, test_mid = train_test_split(labels.measurement_id.unique(), random_state=1)
train_mid, valid_mid = train_test_split(train_mid, random_state=1)
all_mid = sorted(labels.measurement_id)
train_indices  = [all_mid.index(train_m) for train_m in train_mid]
valid_indices  = [all_mid.index(train_m) for train_m in valid_mid]
test_indices  = [all_mid.index(train_m) for train_m in test_mid]

In [13]:
train_data = get_batched_dataset("/n/scratch2/beat_pd_ms_tmp/all_data.tfr", m_ids=train_indices, batch_size=128)
valid_data = get_batched_dataset("/n/scratch2/beat_pd_ms_tmp/all_data.tfr", m_ids=valid_indices, batch_size=256)
test_data = get_batched_dataset("/n/scratch2/beat_pd_ms_tmp/all_data.tfr", m_ids=test_indices, batch_size=256)

In [ ]:
train_data

In [20]:
num_cnn_layers = 2
num_lstm_layers = 2
dropout = 0.5
lin_h=16*40
lstm_h = 4
inputLayer = tf.keras.layers.Input((1500, 3))
x = inputLayer
x = tf.keras.layers.GaussianNoise(0.01)(x)


for i in range(num_cnn_layers):
    x = tf.keras.layers.Conv1D(2*(i+1)+1, (3,), padding="same")(x)
    x = tf.keras.layers.LeakyReLU()(x)
    x = tf.keras.layers.MaxPool1D((2,))(x)
    x = tf.keras.layers.BatchNormalization()(x)

for j in range(num_lstm_layers):
    x = tf.keras.layers.CuDNNLSTM(int(lstm_h/(2**j)), return_sequences=True)(x)
    x = tf.keras.layers.LeakyReLU()(x)
    x = tf.keras.layers.BatchNormalization()(x)

x_shape = x.shape
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(lin_h)(x)
x = tf.keras.layers.LeakyReLU(name="encoder_output")(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(x_shape[1].value * x_shape[2].value)(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.Reshape((x_shape[1].value, x_shape[2].value))(x)

for j in range(num_lstm_layers):
    x = tf.keras.layers.CuDNNLSTM(int(lstm_h/(2**(num_lstm_layers-j))), return_sequences=True)(x)
    x = tf.keras.layers.LeakyReLU()(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
for i in range(num_cnn_layers):
    x = tf.keras.layers.Conv1D(2*num_cnn_layers-2*(i)+1, (3,), padding="same")(x)
    x = tf.keras.layers.LeakyReLU()(x)
    x = tf.keras.layers.UpSampling1D(2)(x)
    x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv1D(3, (3,), padding="same")(x)
model = tf.keras.models.Model(inputs=inputLayer, outputs=x)
model.summary()
model.compile("adam", loss="logcosh", metrics=["mean_squared_error"])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1500, 3)           0         
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 1500, 3)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 1500, 3)           30        
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 1500, 3)           0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 750, 3)            0         
_________________________________________________________________
batch_normalization_v1_10 (B (None, 750, 3)            12        
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 750, 5)            50        
__________

In [17]:
import numpy as np

In [ ]:
model.predict(np.random.random((1,1500,3)))

In [15]:
x, y = None, None
with tf.Session() as sess:
    x, y = train_data.take(10).make_one_shot_iterator().get_next()
    y = y.eval()
    x = x.eval()

In [18]:
np.isnan(x).any()

False

In [19]:
earlyStopping = tf.keras.callbacks.EarlyStopping(patience=20, verbose=True)
modelCheckpoint = tf.keras.callbacks.ModelCheckpoint("/n/scratch2/ms994/autoEncoderCISPD.h5", save_best_only=True)
history = model.fit(train_data, steps_per_epoch=1000, epochs=200, validation_data=valid_data, validation_steps=100, callbacks=[earlyStopping, modelCheckpoint],)

Instructions for updating:
Use tf.cast instead.
Epoch 1/200
1000/1000 [==============================] - 370s 370ms/step - loss: 0.1271 - mean_squared_error: 0.7651 - val_loss: 0.0773 - val_mean_squared_error: 0.1680
Epoch 2/200
1000/1000 [==============================] - 361s 361ms/step - loss: 0.0640 - mean_squared_error: 0.1391 - val_loss: 0.0771 - val_mean_squared_error: 0.1714
Epoch 3/200
1000/1000 [==============================] - 362s 362ms/step - loss: 0.0413 - mean_squared_error: 0.0894 - val_loss: 0.0338 - val_mean_squared_error: 0.0725
Epoch 4/200
1000/1000 [==============================] - 361s 361ms/step - loss: 0.0315 - mean_squared_error: 0.0679 - val_loss: 0.0294 - val_mean_squared_error: 0.0628
Epoch 5/200
1000/1000 [==============================] - 361s 361ms/step - loss: 0.0283 - mean_squared_error: 0.0610 - val_loss: 0.0288 - val_mean_squared_error: 0.0615
Epoch 6/200
1000/1000 [==============================] - 361s 361ms/step - loss: 0.0268 - mean_squared_erro

KeyboardInterrupt: 